In [41]:
import pandas as pd
import numpy as np

In [42]:
train = pd.read_csv('train.csv', index_col='PassengerId')

In [43]:
X = pd.DataFrame()
y = train['Survived']

Copy across numeric fields without missing data

In [44]:
X['Pclass'] = train['Pclass'] # ordinal
X['SibSp'] = train['SibSp']
X['Parch'] = train['Parch']

Convert Sex to categorical

In [45]:
X['Female'] = np.where(train['Sex'] == 'female', 1, 0)

Extract Title from Name

In [61]:
title = train['Name'].map(lambda x: \
                  (x.split(',')[1].split(' ')[2] if ' the ' in x \
                    else x.split(',')[1].split(' ')[1]) if (',' in x) \
                  else x.split(' ')[1]) 
X['Title_Mr'] = title.isin(['Mr.']).astype(int)
X['Title_Mrs'] = title.isin(['Mrs.', 'Mlle.']).astype(int)
X['Title_Miss'] = title.isin(['Miss.', 'Mme', 'Ms.']).astype(int)
X['Title_Miss'] = title.isin(['Master.']).astype(int)
X['title_Other'] = ((X['Title_Mr'] + X['Title_Mrs'] + X['Title_Miss'] + X['Title_Master']) == 0).astype(int)


array(['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.',
       'Ms.', 'Major.', 'Lady.', 'Sir.', 'Mlle.', 'Col.', 'Capt.',
       'Countess.', 'Jonkheer.'], dtype=object)

Predict missing Age

In [52]:
X_with = X[ train['Age'].notnull() ]
X_without = X[ train['Age'].isnull() ]

from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg.fit(X_with, train[ train['Age'].notnull() ]['Age'])

X['Age'] = train['Age']
X.loc[ train['Age'].isnull(), 'Age'] = reg.predict(X_without)
X['Age'] = X['Age'].astype(int)